# Build the logbook for Hologram with empty or filters

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : September 19th 2022
- update : September 21th 2022


Works directory per directory (day by day)

CCIN2P3:

- works with kernels **anaconda3_auxtel** (with libradtran) at CC
- works with kernel **python 3** locally

In [1]:
import sys
sys.path

['/pbs/throng/lsst/users/dagoret/desc/AuxTelComm/notebookccdm_gen3/runspectractor_standalone/2022_09',
 '',
 '/opt/conda/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python38.zip',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/lib-dynload',
 '/pbs/home/d/dagoret/.local/lib/python3.8/site-packages',
 '/pbs/throng/lsst/users/dagoret/desc/firecrown',
 '/pbs/throng/lsst/users/dagoret/desc/augur',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/Deprecated-1.2.11-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/docopt-0.6.2-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/mimeparse-0.1.3-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/rail-0.1.dev0-py3.8.eg

In [2]:
! ls /sps/lsst/groups/auxtel/data/2022/BG40~holo4_003/20220629

exposure_2022062900054_postisrccd.fits	exposure_2022062900325_postisrccd.fits
exposure_2022062900055_postisrccd.fits	exposure_2022062900327_postisrccd.fits
exposure_2022062900056_postisrccd.fits	exposure_2022062900328_postisrccd.fits
exposure_2022062900084_postisrccd.fits	exposure_2022062900359_postisrccd.fits
exposure_2022062900110_postisrccd.fits	exposure_2022062900360_postisrccd.fits
exposure_2022062900113_postisrccd.fits	exposure_2022062900362_postisrccd.fits
exposure_2022062900137_postisrccd.fits	exposure_2022062900388_postisrccd.fits
exposure_2022062900138_postisrccd.fits	exposure_2022062900389_postisrccd.fits
exposure_2022062900139_postisrccd.fits	exposure_2022062900414_postisrccd.fits
exposure_2022062900164_postisrccd.fits	exposure_2022062900415_postisrccd.fits
exposure_2022062900165_postisrccd.fits	exposure_2022062900416_postisrccd.fits
exposure_2022062900166_postisrccd.fits	exposure_2022062900446_postisrccd.fits
exposure_2022062900201_postisrccd.fits	exposure_2022062900449_po

# Select the combination DATE, FILTER, DISPERSER

In [3]:
DATE = '20220630'
filterdispersernames = ["empty~holo4_003","BG40~holo4_003","FELH0600~holo4_003","SDSSg~holo4_003",\
                        "empty~ronchi170lpmm","BG40~ronchi170lpmm","FELH0600~ronchi170lpmm","SDSSg~ronchi170lpmm",\
                       ]
filterdispersername = filterdispersernames[2]
version="v5.0"  # new orientation

# If position need to be specified add it in this dictionnary

In [4]:
# reconstructed / good /xpos /ypos

if version == "v5.0":
    info={\

          # with red filter
          'exposure_2022031600351_postisrccd.fits':(1,1,300.,1700),\
         }
        

# Imports

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re

plt.rcParams["figure.figsize"] = (16,4)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [6]:
from astropy.io import fits

In [7]:
import imp
MODULE_EXTENSIONS = ('.py', '.pyc', '.pyo')

/tmp/ipykernel_3316/3547059130.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [8]:
def package_contents(package_name):
    file, pathname, description = imp.find_module(package_name)
    if file:
        raise ImportError('Not a package: %r', package_name)
    # Use a set because some may be both source and compiled.
    return set([os.path.splitext(module)[0]
        for module in os.listdir(pathname)
        if module.endswith(MODULE_EXTENSIONS)])

In [9]:
package_contents("spectractor")

{'__init__',
 '_version',
 'astrometry',
 'config',
 'logbook',
 'parameters',
 'tools'}

In [10]:
package_contents("spectractor/extractor")

{'__init__',
 'background',
 'chromaticpsf',
 'dispersers',
 'extractor',
 'images',
 'psf',
 'spectroscopy',
 'spectrum',
 'targets'}

# Initialisation

In [11]:
dir_logbooks="logbooks"

In [12]:
import os
if not os.path.exists(dir_logbooks):
    os.makedirs(dir_logbooks)

## Get inputs files and sort them according date and number

In [13]:
home=os.getenv("HOME")

In [14]:
image_path=filterdispersername
logbook_csv=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_"+ DATE +"_" + version + ".csv")
logbook_xlsx=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_" +DATE +"_" + version + ".xlsx")

# Path

In [15]:
HOSTCC=True
LAPTOP=False

In [16]:
# Set path depending on which computer running (according HOSTCC)
if HOSTCC:
    path_auxtel="/sps/lsst/groups/auxtel"
    path_spectractor=os.path.join(path_auxtel,"softs/github/desc/Spectractor")
    path_spectractor_config=os.path.join(path_spectractor,"config")
    path_images=os.path.join(path_auxtel,"data/2022/"+filterdispersername+"/"+DATE)

else:
    if not LAPTOP:
        path_auxtel="/Users/sylvie/DATA/AuxTelData2022"
        path_spectractor=os.path.join(path_auxtel,"/Users/sylvie/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        path_images=os.path.join(path_auxtel,"data/2022/"+filterdispersername+"/"+DATE)
        
    else:
        path_auxtel="/Users/dagoret/DATA/AuxTelData2022"
        path_spectractor=os.path.join(path_auxtel,"/Users/dagoret/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        path_images=os.path.join(path_auxtel,"data/2022/"+filterdispersername+"/"+DATE)
        

# Check the images exists

In [17]:
dir_images=path_images

In [18]:
all_files=os.listdir(dir_images)

In [19]:
all_files

['exposure_2022063000480_postisrccd.fits',
 'exposure_2022063000566_postisrccd.fits',
 'exposure_2022063000594_postisrccd.fits',
 'exposure_2022063000049_postisrccd.fits',
 'exposure_2022063000395_postisrccd.fits',
 'exposure_2022063000064_postisrccd.fits',
 'exposure_2022063000455_postisrccd.fits',
 'exposure_2022063000220_postisrccd.fits',
 'exposure_2022063000080_postisrccd.fits',
 'exposure_2022063000296_postisrccd.fits',
 'exposure_2022063000540_postisrccd.fits',
 'exposure_2022063000440_postisrccd.fits',
 'exposure_2022063000481_postisrccd.fits',
 'exposure_2022063000081_postisrccd.fits',
 'exposure_2022063000182_postisrccd.fits',
 'exposure_2022063000454_postisrccd.fits',
 'exposure_2022063000126_postisrccd.fits',
 'exposure_2022063000235_postisrccd.fits',
 'exposure_2022063000541_postisrccd.fits',
 'exposure_2022063000356_postisrccd.fits',
 'exposure_2022063000526_postisrccd.fits',
 'exposure_2022063000525_postisrccd.fits',
 'exposure_2022063000409_postisrccd.fits',
 'exposure_

In [20]:
all_selected_files = []
for filename in all_files:
    if re.search(".fits$",filename):
        all_selected_files.append(filename)
N=len(all_selected_files)

In [21]:
all_selected_files=np.array(all_selected_files)

### Sort files

In [22]:
def file_tag_forsorting(filename):
    #m=re.findall('^Cor_holo4_003_.*([0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]_.*)[.]fits$',filename)
    m=re.findall('exposure.*_(.*)_postisrccd[.]fits$',filename)
    
    return m[0]

In [23]:
file_tag_forsorting('exposure_2022031700310_postisrccd.fits')

'2022031700310'

In [24]:
#all_tags = [ file_tag_forsorting(filename)[0] for filename in all_selected_files]
all_tags = [ file_tag_forsorting(filename) for filename in all_selected_files]
all_tags=np.array(all_tags)
all_tags

array(['2022063000480', '2022063000566', '2022063000594', '2022063000049',
       '2022063000395', '2022063000064', '2022063000455', '2022063000220',
       '2022063000080', '2022063000296', '2022063000540', '2022063000440',
       '2022063000481', '2022063000081', '2022063000182', '2022063000454',
       '2022063000126', '2022063000235', '2022063000541', '2022063000356',
       '2022063000526', '2022063000525', '2022063000409', '2022063000593',
       '2022063000125', '2022063000181', '2022063000511', '2022063000394',
       '2022063000357', '2022063000321', '2022063000295', '2022063000280',
       '2022063000065', '2022063000265', '2022063000161', '2022063000236',
       '2022063000565', '2022063000341', '2022063000140', '2022063000050',
       '2022063000410', '2022063000111', '2022063000495'], dtype='<U13')

In [25]:
indexes=np.argsort(all_tags)

In [26]:
sorted_files=all_selected_files[indexes]

In [27]:
sorted_files

array(['exposure_2022063000049_postisrccd.fits',
       'exposure_2022063000050_postisrccd.fits',
       'exposure_2022063000064_postisrccd.fits',
       'exposure_2022063000065_postisrccd.fits',
       'exposure_2022063000080_postisrccd.fits',
       'exposure_2022063000081_postisrccd.fits',
       'exposure_2022063000111_postisrccd.fits',
       'exposure_2022063000125_postisrccd.fits',
       'exposure_2022063000126_postisrccd.fits',
       'exposure_2022063000140_postisrccd.fits',
       'exposure_2022063000161_postisrccd.fits',
       'exposure_2022063000181_postisrccd.fits',
       'exposure_2022063000182_postisrccd.fits',
       'exposure_2022063000220_postisrccd.fits',
       'exposure_2022063000235_postisrccd.fits',
       'exposure_2022063000236_postisrccd.fits',
       'exposure_2022063000265_postisrccd.fits',
       'exposure_2022063000280_postisrccd.fits',
       'exposure_2022063000295_postisrccd.fits',
       'exposure_2022063000296_postisrccd.fits',
       'exposure_202

In [28]:
N=len(sorted_files)

# Build the logbook from the headers

In [29]:
idx=0
#ccdir='/sps/lsst/groups/auxtel/data/2021/holo/FlipCleans'
ccdir=dir_images
all_dates=[]
all_files=[]
all_seqnum=[]
all_subdir=[]
all_indexes=[]
all_targets=[]
all_filters=[]
all_disperser=[]
all_airmass=[]
all_exposures=[]
all_focus=[]
all_seeing=[]
all_P=[]
all_T=[]
all_H=[]
all_W=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header 
    all_indexes.append(idx)
    all_seqnum.append(hdr0['SEQNUM'])
    all_files.append(filename)
    all_subdir.append(ccdir)
    targetname=hdr0['OBJECT']
    all_targets.append(targetname)
    all_dates.append(hdr0['DATE-OBS'])
    all_airmass.append(hdr0['AMSTART'])
    all_filters.append(hdr0['FILTER'])
    all_disperser.append(hdr0['GRATING'])
    all_exposures.append(hdr0['EXPTIME'])
    all_focus.append(hdr0['FOCUSZ'])
    all_seeing.append(hdr0['SEEING'])
    all_P.append(hdr0['PRESSURE'])
    all_T.append(hdr0['AIRTEMP'])
    all_H.append(hdr0['HUMIDITY'])
    all_W.append(hdr0['WINDSPD'])
       
    
    idx+=1
    

# Make pandas dataframe

## Create pandas dataframe

In [30]:
df=pd.DataFrame()

In [31]:
len(all_airmass)

43

In [32]:
N=len(all_indexes)

## Fill with container info

In [33]:
df['index']=all_indexes
df['date']=all_dates
df['path']=all_subdir
df['file']=all_files
df['num']=all_seqnum
df['object']=all_targets
df['filter']=all_filters
df['disperser']=all_disperser
df['airmass']=all_airmass
df['exposure']=all_exposures
df['focus']=all_focus
df['seeing']=all_seeing
df['pressure']=all_P
df['temperature']=all_T
df['humidity']=all_H
df['windspd']=all_W

## Add dictionnary info

### Initialise new columns

In [34]:
Obj_posXpix=np.full(N,np.nan)
Obj_posYpix=np.full(N,np.nan)
Dx=np.full(N,np.nan)
Dy=np.full(N,np.nan)
run=np.full(N,np.nan)
quality=np.full(N,np.nan)
skip=np.full(N,"    ")

### Fill dictionnary info

In [35]:
# loop on rows index
for idx in np.arange(N):
    filename=all_files[idx]
    # check if filename is in dictionnary
    if filename in info.keys():
        (run_flag,quality_flag,xpos,ypos)=info[filename]
        Obj_posXpix[idx]=int(xpos)
        Obj_posYpix[idx]=int(ypos)
        run[idx]=run_flag
        if quality_flag==1:
            skip[idx] = "    "
            quality[idx]=1
        else:
            skip[idx] = "skip"
            quality[idx]=0

In [36]:
df['Obj-posXpix']=Obj_posXpix
df['Obj-posYpix']=Obj_posYpix
df['Dx']=Dx
df['Dy']=Dy
df['skip']=skip
df['run']=run
df['quality']=quality

In [37]:
pd.set_option('display.max_rows', None)

In [38]:
df

,index,date,path,file,num,object,filter,disperser,airmass,exposure,...,temperature,humidity,windspd,Obj-posXpix,Obj-posYpix,Dx,Dy,skip,run,quality
0,0,2022-06-30T23:17:36.488,/sps/lsst/groups/auxtel/data/2022/FELH0600~hol...,exposure_2022063000049_postisrccd.fits,49,HD142331,FELH0600,holo4_003,1.392998,30.0,...,7.2,21.0,4.8,NaN,NaN,NaN,NaN,,NaN,NaN
1,1,2022-06-30T23:18:19.508,/sps/lsst/groups/auxtel/data/2022/FELH0600~hol...,exposure_2022063000050_postisrccd.fits,50,HD142331,FELH0600,holo4_003,1.389573,30.0,...,7.2,21.0,4.7,NaN,NaN,NaN,NaN,,NaN,NaN
2,2,2022-06-30T23:27:28.364,/sps/lsst/groups/auxtel/data/2022/FELH0600~hol...,exposure_2022063000064_postisrccd.fits,64,HD142331,FELH0600,holo4_003,1.348513,30.0,...,7.2,21.0,3.9,NaN,NaN,NaN,NaN,,NaN,NaN
3,3,2022-06-30T23:28:11.267,/sps/lsst/groups/auxtel/data/2022/FELH0600~hol...,exposure_2022063000065_postisrccd.fits,65,HD142331,FELH0600,holo4_003,1.345487,30.0,...,7.2,21.0,4.4,NaN,NaN,NaN,NaN,,NaN,NaN
4,4,2022-06-30T23:38:47.453,/sps/lsst/groups/auxtel/data/2022/FELH0600~hol...,exposure_2022063000080_postisrccd.fits,80,HD185975,FELH0600,holo4_003,2.022890,30.0,...,7.2,21.0,4.4,NaN,NaN,NaN,NaN,,NaN,NaN
5,5,2022-06-30T23:39:30.530,/sps/lsst/groups/auxtel/data/2022/FELH0600~hol...,exposure_2022063000081_postisrccd.fits,81,HD185975,FELH0600,holo4_003,2.022419,30.0,...,7.1,21.0,6.4,NaN,NaN,NaN,NaN,,NaN,NaN
6,6,2022-07-01T00:00:25.220,/sps/lsst/groups/auxtel/data/2022/FELH0600~hol...,exposure_2022063000111_postisrccd.fits,111,HD142331,FELH0600,holo4_003,1.232307,30.0,...,6.9,22.0,5.2,NaN,NaN,NaN,NaN,,NaN,NaN
7,7,2022-07-01T00:09:33.448,/sps/lsst/groups/auxtel/data/2022/FELH0600~hol...,exposure_2022063000125_postisrccd.fits,125,HD142331,FELH0600,holo4_003,1.207383,30.0,...,6.6,22.0,6.7,NaN,NaN,NaN,NaN,,NaN,NaN
8,8,2022-07-01T00:10:16.352,/sps/lsst/groups/auxtel/data/2022/FELH0600~hol...,exposure_2022063000126_postisrccd.fits,126,HD142331,FELH0600,holo4_003,1.205547,30.0,...,6.6,23.0,6.0,NaN,NaN,NaN,NaN,,NaN,NaN
9,9,2022-07-01T00:19:29.453,/sps/lsst/groups/auxtel/data/2022/FELH0600~hol...,exposure_2022063000140_postisrccd.fits,140,HD142331,FELH0600,holo4_003,1.183295,30.0,...,6.4,23.0,5.3,NaN,NaN,NaN,NaN,,NaN,NaN


# Make logbook


In [39]:
N=len(df)
N

43

In [40]:
if N>0:
    df.to_excel(logbook_xlsx)
    df.to_csv(logbook_csv)

In [41]:
logbook_csv

'logbooks/auxtellogbook_FELH0600~holo4_003_20220630_v5.0.csv'